In [ ]:
%matplotlib inline

from wordcloud import WordCloud, STOPWORDS
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.cluster import KMeans
import numpy as np
from collections import Counter
import nltk, string
from nltk.corpus import stopwords
import spacy  # you need to install the language model this way: python -m spacy download en_core_web_md

nlp = spacy.load("en_core_web_md")

stop = stopwords.words("english")

exclude = set(string.punctuation)

In [ ]:
def get_closest_to_centroid(vectors, centroid):
    rank = [
        [
            x,
            cosine_similarity(
                centroid.reshape(1, -1), vectors[x].reshape(1, -1)
            )[0][0],
        ]
        for x in range(len(vectors))
    ]
    rank.sort(key=lambda x: x[1], reverse=True)
    closest = rank[:3]
    return closest


def clean_text(text):
    text = text.lower()
    text = "".join(ch for ch in text if ch not in exclude)
    text = [x for x in nltk.word_tokenize(text) if x not in stop]
    return text

In [ ]:
df = pd.read_csv("data/mybinder.org-user-survey-feb-2020.csv")

answer_embs = []

for index, row in df.iterrows():
    answer = row[
        "If you could change one thing about Binder, what would it be?"
    ]
    if type(answer) is str and len(answer) > 3:
        answer_emb = nlp(answer).vector
        answer_embs.append([index, answer, answer_emb])

comment_words = " ".join([x[1] for x in answer_embs])

wordcloud = WordCloud(
    width=800, height=800, background_color="white", min_font_size=10
).generate(comment_words)

plt.figure(figsize=(8, 8), facecolor=None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
n_clusters = 10

X = np.array([x[2] for x in answer_embs])
kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(X)
clusters = {
    x: {"centroid": kmeans.cluster_centers_[x], "answers": [], "vectors": []}
    for x in range(len(kmeans.cluster_centers_))
}

for x in range(len(answer_embs)):
    answer = answer_embs[x][1]
    vector = answer_embs[x][2]
    label = kmeans.labels_[x]
    clusters[label]["answers"].append(answer)
    clusters[label]["vectors"].append(vector)

In [ ]:
for cluster, infos in clusters.items():
    answers = infos["answers"]
    vectors = infos["vectors"]
    centroid = infos["centroid"]
    comment_words = [word for answer in answers for word in clean_text(answer)]
    most_common = Counter(comment_words).most_common(5)
    central_ids = get_closest_to_centroid(vectors, centroid)
    central_answers = [answers[id_[0]] for id_ in central_ids]
    for answ in central_answers:
        print(answ)
    print(
        "---> other",
        len(answers) - len(central_answers),
        "messages around the same topic",
    )

    #    for answ in answers:
    #        print (answ)
    print(" ")

In [ ]:
def search_query(q, text):
    # if multiword query
    if len(q.split(" ")) > 1:
        if q.lower() in answer.lower():
            return True
    else:
        # we lowercase and remove puctuation
        # then search for a perfect match
        text = clean_text(text)
        if q.lower() in text:
            return True

In [ ]:
q = "R"

for index, row in df.iterrows():
    answer = row[
        "If you could change one thing about Binder, what would it be?"
    ]
    if type(answer) is str and len(answer) > 3:
        check_relevance = search_query(q, answer)
        if check_relevance is True:
            print(answer + "\n")